# Cutting Stock Problem

# Maryam Hoseeinali 610398209

## general explanation:

### In this implementation the minimum amount of wastage and needed role has been calculated based on the Genetic, Simulated Annealing, and Hill Climbing Algorithms .



## `Simulated Annealing Algorithm:`

In [95]:
import random
import copy
from numpy import exp
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Define'Problem' class to represent the problem instance
class Problem:
    def __init__(self, length, orders):
        self.length = length
        self.orders = orders

# Define 'Plan' class to represent a solution plan
class Plan:
    def __init__(self, problem):
        self.problem = problem
        self.orders_on_rolls = None
        self.space_left = None
        self.waste = None

    # Generate a random initial solution plan
    def random_plan(self):
        self.orders_on_rolls = [[]]
        self.space_left = [self.problem.length]

        for order_id, order in enumerate(self.problem.orders):
            if self.space_left[-1] >= order:
                self.orders_on_rolls[-1].append(order_id)
                self.space_left[-1] -= order
            else:
                self.orders_on_rolls.append([order_id])
                self.space_left.append(self.problem.length - order)

        self.calc_waste()

    # Calculate and store the waste in the current solution plan
    def calc_waste(self):
        self.waste = sum(self.space_left)
        return self.waste

    # Generate a new solution plan by moving orders between rolls.
    def new_plan(self):
        new_plan = Plan(self.problem)
        new_plan.orders_on_rolls = copy.deepcopy(self.orders_on_rolls)
        new_plan.space_left = copy.deepcopy(self.space_left)

        change = False
        while not change:
            chosen_roll = random.choice(range(len(new_plan.space_left)))
            order_id = random.choice(new_plan.orders_on_rolls[chosen_roll])

            for other_roll in range(len(new_plan.orders_on_rolls)):
                if other_roll == chosen_roll:
                    continue
                if new_plan.space_left[other_roll] >= self.problem.orders[order_id]:
                    new_plan.move_order(order_id, chosen_roll, other_roll)
                    change = True
                    break

            if len(new_plan.orders_on_rolls[chosen_roll]) == 0:
                new_plan.remove_roll(chosen_roll)

        new_plan.calc_waste()
        return new_plan

    # Move an order from one roll to another.
    def move_order(self, order_id, from_roll, to_roll):
        self.space_left[to_roll] -= self.problem.orders[order_id]
        self.orders_on_rolls[to_roll].append(order_id)
        self.orders_on_rolls[from_roll].remove(order_id)
        self.space_left[from_roll] += self.problem.orders[order_id]

    # Remove an empty roll from the solution.
    def remove_roll(self, roll):
        del self.orders_on_rolls[roll]
        del self.space_left[roll]

    # Return a list representing the orders on each roll in the solution.
    def show_orders(self):
        return [[self.problem.orders[order_id] for order_id in roll] for roll in self.orders_on_rolls]

# Simulated annealing algorithm to optimize the solution.
def simulated_annealing(problem, iterations, temp, show=True):
    waste_amount = []  # Tracks the waste amount over iterations
    current = Plan(problem)
    current.random_plan()
    waste_amount.append(current.waste)

    best = current
    for i in range(iterations):
        new = current.new_plan()
        if new.waste < best.waste:
            best = new
            waste_amount.append(best.waste)

        diff = new.waste - current.waste
        t = temp / float(i + 1)
        if diff < 0 or random.random() < exp(-diff / t):
            current = new

    if show:
        print(f"Rolls used: {len(best.orders_on_rolls)}")
        print(f"Fianl Solution: {best.show_orders()}")
        print(f"Waste: {best.waste}")
    else:
        print(f"Final: Rolls used: {len(best.orders_on_rolls)}, Waste: {best.waste}")

    return waste_amount


### `Overall Functionality:`


This code try to solve the Cutting Stock Problem by finding an optimal allocation of orders to minimize waste.
It uses the Simulated Annealing algorithm, The algorithm starts with a random solution, explores neighboring solutions through order movements, and gradually converges towards better solutions. The Metropolis criterion controls the acceptance of solutions with worse waste values.


### `Problem` Class:

 The `Problem` class represents the problem instance:
 - `Problem`: Represents the length of the rolls.
- ` orders`: A list of integers representing the different orders.

### `Plan` Class:

The `Plan` class represents a solution plan:

- `random_plan`: Generates a random initial solution plan by allocating orders to rolls. It minimizes waste by efficiently filling the rolls.

- `calc_waste`: Calculates and stores the waste in the current solution plan, which is the total unused capacity of the rolls.

- `new_plan`: Generates a new solution plan by moving orders between rolls. It tries to explore neighboring solutions efficiently.

- `move_order`: Moves an order from one roll to another, updating the allocation and available capacity of both rolls.

- `remove_roll`: Removes an empty roll from the solution.

- `show_orders`: Returns a list representing the orders on each roll in the solution.

### `simulated_annealing` Function:

The `simulated_annealing` function implements the Simulated Annealing algorithm to optimize the solution plan. It iteratively explores and improves the solution by accepting or rejecting neighboring solutions:

- Initialization: It starts with a random initial solution and tracks the waste amount over iterations.

- Main Loop: It iterates for a specified number of `iterations` and generates new solution plans (`new`) by moving orders between rolls.

- Acceptance Criteria: It evaluates whether to accept a new solution based on the Metropolis criterion. Better solutions are always accepted, and worse solutions are accepted with a probability controlled by temperature (`temp`).

- Temperature Control: The temperature (`temp`) decreases over iterations, reducing the likelihood of accepting worse solutions as the algorithm progresses.

- Output: The function can display the final solution, including the number of rolls used, the Final Solution, and the waste.





### `Input Handling and Testing the Simulated Annealing Algorithm:`







In [98]:
f = open('input1.stock').readlines()
lens = int(f[0].split()[2])
reqs = list(map(int, f[3].replace(' ','').split(',')))
problem = Problem(lens, reqs)
scores = simulated_annealing(problem, 10000, 30)

Rolls used: 51
Fianl Solution: [[75, 18, 46, 53, 45, 106, 43, 92, 123, 368], [914, 23], [106, 79, 248, 149, 116, 69, 71, 92, 61], [106, 135, 107, 149, 33, 224, 148, 80], [119, 60, 149, 88, 70, 144, 99, 117, 106], [460, 241, 118, 78], [716, 109, 170], [171, 337, 354, 86], [662, 230], [549, 84, 125, 126], [618, 88, 180], [805, 186], [405, 333, 246], [753, 145], [463, 211, 266], [555, 402], [557, 312, 115], [967], [788, 187], [501, 495], [988], [592, 370], [672, 301], [525, 283, 181], [515, 351, 125], [632, 315], [933], [506, 278], [648, 312], [753, 232], [678, 249], [544, 424], [457, 218, 280], [365, 284, 292], [653, 251], [788], [686, 266], [627, 356], [609, 346], [689, 264], [987], [868], [409, 295, 286], [441, 518], [517, 412], [557, 437], [557, 268], [507, 414], [660, 306], [581, 286], [371, 532]]
Waste: 2928


In [99]:

f = open('input2.stock').readlines()
lens = int(f[0].split()[2])
reqs = list(map(int, f[3].replace(' ','').split(',')))
problem = Problem(lens, reqs)
scores = simulated_annealing(problem, 10000, 100)

Rolls used: 78
Fianl Solution: [[1820, 1560, 2100], [1880, 2050, 1560], [2200, 1880, 1520], [2100, 1560, 1520], [1880, 2140, 1380], [2100, 1560, 1880], [2200, 1930, 1380], [2150, 1930, 1520], [1380, 1520, 2140], [2150, 1520, 1930], [2150, 1930, 1520], [2140, 1560, 1820], [1880, 1520, 2200], [1930, 2100, 1380], [2150, 1380, 2000], [2140, 1880, 1520], [1930, 1820, 1520], [1880, 2150, 1380], [2000, 1820, 1560], [2140, 1880, 1380], [2200, 1820, 1520], [2050, 2050, 1380], [2150, 1560, 1820], [2150, 2000, 1380], [2050, 1380, 2150], [2100, 1880, 1520], [2140, 1820, 1560], [2200, 1520, 1880], [2100, 1930, 1380], [1520, 2150, 1880], [1380, 1710, 2000], [2140, 1380, 1560], [1820, 2150, 1380], [1380, 2050, 1380], [1820, 2200, 1560], [2150, 1930, 1520], [2200, 1930, 1380], [2200, 1820, 1380], [2200, 1930, 1380], [2050, 2150, 1380], [2200, 1520, 1520], [2200, 1820, 1520], [1520, 2140, 1930], [2100, 2050], [2200, 1380, 1930], [2140, 1520, 1520], [1880, 2150, 1520], [2140, 1880, 1560], [2050, 1710, 1

In [100]:
f = open('input3.stock').readlines()
lens = int(f[0].split()[2])
reqs = list(map(int, f[3].replace(' ','').split(',')))
problem = Problem(lens, reqs)
scores = simulated_annealing(problem, 10000, 30)

Rolls used: 98
Fianl Solution: [[6, 288, 19, 1, 9, 1, 2, 5, 9, 19, 2, 7, 16, 1, 13, 3, 4, 3, 1, 1, 2, 1, 12, 2, 5, 2, 1, 1, 1, 2, 1, 2, 2, 4, 3, 1, 1, 3, 1, 6, 3, 2, 1, 1, 1, 3, 5, 5, 3, 1, 2, 1, 1, 2, 1, 3, 2], [470, 1, 3, 9, 1, 1, 2, 3, 9, 1], [16, 3, 2, 1, 5, 1, 2, 7, 10, 10, 8, 8, 7, 1, 1, 1, 3, 4, 1, 3, 1, 1, 1, 1, 5, 13, 5, 10, 160, 3, 3, 1, 1, 1, 6, 5, 3, 3, 5, 19, 6, 1, 6, 6, 85, 6, 2, 2, 2, 2, 1, 7, 1, 8, 2, 6, 2, 4, 6, 1], [2, 5, 1, 134, 1, 4, 1, 2, 12, 91, 50, 9, 2, 1, 3, 2, 2, 10, 1, 3, 1, 2, 2, 1, 4, 93, 6, 18, 18, 3, 1, 4, 7, 3], [5, 8, 244, 8, 134, 2, 10, 4, 7, 13, 5, 1, 1, 2, 4, 5, 3, 1, 7, 1, 1, 7, 3, 2, 4, 5, 11, 1], [88, 11, 8, 3, 6, 2, 1, 12, 1, 1, 5, 6, 1, 1, 7, 3, 2, 13, 29, 35, 1, 2, 3, 3, 3, 1, 7, 4, 1, 1, 3, 3, 3, 2, 4, 1, 5, 2, 2, 5, 11, 92, 7, 7, 12, 8, 54, 4, 8, 4, 2], [4, 2, 1, 6, 5, 5, 4, 10, 170, 2, 2, 7, 9, 97, 16, 6, 6, 5, 10, 16, 1, 2, 3, 26, 1, 2, 8, 3, 1, 1, 3, 7, 3, 6, 2, 7, 6, 8, 8, 4, 1, 7, 1, 2], [5, 9, 10, 4, 9, 1, 1, 7, 41, 5, 1, 13, 5, 3, 2, 2

In [103]:
f = open('input4.stock').readlines()
lens = int(f[0].split()[2])
reqs = list(map(int, f[3].replace(' ','').split(',')))
problem = Problem(lens, reqs)
scores = simulated_annealing(problem, 10000, 30)

Rolls used: 217
Fianl Solution: [[4, 2, 2, 2, 1, 1, 3, 2, 1, 8, 2, 5, 2, 1, 1, 2, 6, 1, 3, 2, 1, 4, 8, 2, 2, 4, 3, 1, 1, 1, 2, 2, 2, 1, 4, 3, 2, 5, 1], [1, 7, 2, 7, 3, 2, 1, 4, 1, 1, 2, 4, 3, 4, 1, 1, 6, 1, 7, 2, 22, 3, 3, 1, 1, 1, 2, 1, 2, 3], [74, 2, 1, 1, 1, 3, 2, 8, 2, 2, 2, 1, 1], [67, 1, 3, 2, 3, 4, 3, 2, 1, 1, 4, 3, 3, 3], [6, 5, 11, 2, 7, 4, 3, 5, 30, 3, 3, 1, 4, 3, 8, 1, 4], [66, 4, 6, 5, 4, 2, 2, 5, 2, 2], [4, 2, 12, 2, 2, 1, 4, 6, 5, 7, 2, 3, 3, 3, 7, 6, 5, 2, 5, 5, 7, 3, 4], [65, 3, 5, 4, 3, 2, 2, 3, 8, 5], [61, 2, 2, 7, 6, 3, 4, 4, 5, 3, 2], [50, 7, 6, 4, 3, 8, 13, 7], [99], [11, 5, 6, 8, 5, 5, 6, 3, 3, 2, 4, 6, 9, 17, 6, 4], [48, 6, 4, 7, 5, 11, 4, 4, 5, 5], [3, 5, 6, 6, 4, 6, 8, 4, 3, 7, 4, 3, 6, 10, 9, 4, 3, 5], [6, 2, 8, 6, 3, 6, 5, 4, 3, 4, 5, 4, 7, 7, 5, 5, 6, 5, 6], [60, 5, 3, 4, 10, 6, 9], [8, 5, 7, 5, 6, 20, 4, 8, 5, 20, 4, 5], [5, 12, 10, 9, 3, 10, 6, 5, 10, 9, 7, 5, 7], [54, 8, 5, 6, 15, 4, 7], [13, 4, 10, 5, 6, 14, 5, 6, 9, 4, 6, 5, 8], [15, 15, 4, 7, 6, 6, 7, 

## `Genetic Algorithm:`

In [108]:
import random
# Generates a solution with random roll allocations
def generate_individual(stock_length, requests):
    solution = []
    pending_requests = requests.copy()

    while pending_requests:
        current_roll = []
        total_length = 0

        random.shuffle(pending_requests)

        for request in pending_requests.copy():
            if total_length + request <= stock_length:
                current_roll.append(request)
                total_length += request
                pending_requests.remove(request)

        solution.append(current_roll)
    
    return solution
# Calculates fitness based on the number of rolls used
def fitness(solution):
    
    return len(solution)
    
# Combines two parent solutions to produce a child solution
def crossover(parent1, parent2):
    split_point = len(parent1) // 2
    child = parent1[:split_point] + parent2[split_point:]
    return child

# Applying random changes to a solution
def mutate(solution, stock_length, requests):
    mutation_chance = 0.3

    while random.random() < mutation_chance:
        if solution and random.random() < 0.5:
            roll_index = random.randint(0, len(solution) - 1)
            if len(solution[roll_index]) > 1:
                i, j = random.sample(range(len(solution[roll_index])), 2)
                solution[roll_index][i], solution[roll_index][j] = solution[roll_index][j], solution[roll_index][i]
        else:
            shuffled_requests = [request for roll in solution for request in roll]
            random.shuffle(shuffled_requests)
            return generate_individual(stock_length, shuffled_requests)

    return solution
# Calculates the total waste for a given solution
def calculate_waste(solution, stock_length):
    total_waste = 0
    for roll in solution:
        used_length = sum(roll)
        waste = stock_length - used_length
        total_waste += waste
    return total_waste
    
# Runs the genetic algorithm to find an optimal solution
def genetic_algorithm(stock_length, requests, population_size=50, generations=100):
    population = [generate_individual(stock_length, requests) for _ in range(population_size)]

    for _ in range(generations):
        population.sort(key=fitness)
        next_gen = population[:2]

        while len(next_gen) < population_size:
            parents = random.sample(population[:10], 2)
            child = mutate(crossover(parents[0], parents[1]), stock_length, requests)
            next_gen.append(child)

        population = next_gen

    best_solution = sorted(population, key=fitness)[0]
    return best_solution






### `Overall Functionality:`
this code implements a Genetic Algorithm trying to minimize waste. This algorithm mimics natural selection processes, evolving solutions over generations to find an optimal allocation of orders to rolls.


### `generate_individual` Function:
Creates a random solution (individual) by allocating orders to rolls.
Ensures efficient usage of roll length to minimize waste.

### `fitness` Function:
Measures the fitness of a solution based on the number of rolls used.
Lower roll usage indicates higher fitness.


### `crossover` Function:
Combines two parent solutions to produce offspring.
The offspring inherits characteristics from both parents, potentially leading to better solutions.

### `mutate` Function:
Introduces random changes to a solution.
helps in exploring new solutions and prevents premature convergence on suboptimal solutions.

### `calculate_waste` Function:
Computes the total waste of a solution.
Essential for assessing the effectiveness of the solution in reducing waste.

### `genetic_algorithm` Function:
Manages the whole Genetic Algorithm process.
Starts with a group of solutions, improves them through choosing the best, mixing them up, and making random changes, and then picks the top solution.
Repeats the process many times to steadily make the solutions better.


### `Input Handling and Testing the Genetic Algorithm:`

In [112]:
f = open('input1.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))
best_solution = genetic_algorithm(stock_length, requests)
number_of_rolls = len(best_solution)
print("Number of rolls used:", number_of_rolls)
print('best_solution:')
print(best_solution)


Number of rolls used: 50
best_solution:
[[88, 506, 405], [557, 306, 80, 46], [106, 370, 251, 268], [18, 117, 678, 180], [463, 365, 45, 33, 92], [53, 678, 251], [312, 337, 187, 148], [409, 402, 125, 60], [23, 145, 43, 246, 525], [86, 69, 69, 662, 109], [232, 424, 88, 149, 88], [264, 312, 119, 92, 115, 79], [266, 660, 70], [368, 346, 230], [592, 312, 75], [301, 653], [149, 99, 689, 61], [211, 266, 224, 249], [788, 135, 71], [116, 301, 518], [278, 286, 264, 149], [284, 672], [224, 266, 441], [106, 544, 337], [333, 123, 532], [78, 126, 686, 106], [248, 187, 248, 84, 211], [933], [107, 517, 283], [967], [118, 689, 126], [241, 555, 106], [463, 495], [170, 788], [181, 149, 532, 115], [517, 441], [460, 515], [365, 627], [501, 437], [292, 284, 356], [557, 437], [283, 286, 412], [371, 371], [507, 457], [354, 609], [653, 286], [515, 292], [632, 356], [914], [914]]


In [110]:
f = open('input2.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))
best_solution = genetic_algorithm(stock_length, requests)
number_of_rolls = len(best_solution)
print("Number of rolls used:", number_of_rolls)
print(best_solution)
print(best_solution)


Number of rolls used: 77
[[1380, 2200, 1930], [1520, 2200, 1520], [1380, 1930, 2200], [1560, 1880, 2150], [1520, 1820, 2200], [1560, 1520, 1930], [1380, 2140, 1380], [1880, 1520, 2000], [1710, 1930, 1710], [2050, 1520, 1710], [1820, 2200, 1380], [1930, 1380, 1520], [2200, 1880, 1520], [1930, 1380, 2140], [2000, 2100, 1380], [2140, 2200], [1520, 2140, 1820], [2000, 1380, 2140], [1710, 2050, 1820], [2200, 1380, 1930], [2050, 2000, 1520], [2150, 2100], [2140, 1560, 1880], [1520, 1880, 2200], [1520, 1930, 2140], [1820, 2200, 1560], [1930, 1520, 1710], [1710, 1820, 1930], [1710, 1520, 2150], [2200, 2150], [2140, 1520, 1880], [2050, 1380, 2140], [1380, 2050, 1560], [1820, 2150, 1560], [1380, 2200, 1930], [1560, 1820, 2100], [1520, 1560, 2140], [2200, 1520, 1520], [1380, 2150, 1880], [2200, 1380, 2000], [2200, 2150], [2100, 1380, 1880], [1520, 2150, 1820], [1710, 2000, 1560], [1880, 2140, 1380], [1820, 1560, 2150], [2000, 1380, 2200], [2150, 1380, 2050], [1380, 2100, 2050], [1520, 2140, 1880]

In [113]:
f = open('input3.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))
best_solution = genetic_algorithm(stock_length, requests)
number_of_rolls = len(best_solution)
print("Number of rolls used:", number_of_rolls)
print(best_solution)
print(best_solution)


Number of rolls used: 94
[[138, 8, 1, 3, 13, 5, 224, 3, 2, 4, 1, 2, 2, 6, 8, 1, 74, 3, 1, 1], [4, 110, 1, 18, 11, 234, 8, 5, 3, 11, 1, 4, 7, 2, 35, 2, 2, 1, 3, 5, 7, 24, 2], [138, 11, 5, 124, 11, 3, 4, 7, 2, 3, 87, 7, 26, 3, 23, 2, 3, 10, 6, 5, 15, 4, 1], [2, 7, 311, 3, 1, 5, 124, 2, 13, 16, 7, 4, 3, 1, 1], [128, 7, 18, 5, 12, 2, 16, 3, 2, 209, 5, 1, 2, 4, 86], [313, 159, 1, 26, 1], [3, 4, 1, 43, 1, 335, 1, 9, 3, 2, 4, 1, 2, 1, 12, 2, 3, 8, 5, 6, 18, 9, 1, 4, 1, 19, 1, 1], [78, 5, 6, 14, 196, 9, 5, 9, 178], [1, 4, 39, 4, 271, 169, 7, 2, 2, 1], [12, 5, 1, 4, 9, 11, 3, 204, 2, 17, 1, 3, 6, 10, 1, 2, 1, 11, 14, 9, 15, 4, 5, 6, 130, 3, 6, 1, 2, 1, 1], [153, 255, 9, 7, 59, 2, 10, 2, 1, 2], [14, 13, 4, 3, 8, 403, 9, 16, 3, 8, 7, 1, 2, 9], [17, 3, 5, 116, 1, 17, 234, 13, 9, 14, 3, 6, 2, 3, 7, 14, 6, 4, 2, 19, 3, 1, 1], [7, 13, 7, 5, 1, 184, 3, 275, 2, 2, 1], [15, 6, 10, 9, 1, 3, 8, 90, 2, 2, 4, 6, 12, 2, 49, 1, 17, 14, 10, 85, 2, 6, 8, 7, 6, 2, 6, 3, 5, 6, 7, 13, 1, 2, 7, 1, 7, 6, 1, 9, 2, 1,

In [114]:
f = open('input4.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))
best_solution = genetic_algorithm(stock_length, requests)
number_of_rolls = len(best_solution)
print("Number of rolls used:", number_of_rolls)
print(best_solution)
print(best_solution)


Number of rolls used: 212
[[98, 2], [5, 5, 36, 33, 11, 1, 9], [3, 40, 2, 9, 4, 7, 7, 28], [58, 8, 12, 4, 6, 11, 1], [7, 3, 10, 51, 2, 1, 4, 7, 6, 1, 2, 6], [27, 8, 1, 8, 33, 6, 8, 9], [38, 9, 11, 10, 6, 7, 5, 5, 6, 2, 1], [15, 13, 18, 46, 1, 3, 3, 1], [99, 1], [5, 29, 5, 12, 11, 25, 2, 4, 4, 3], [1, 29, 8, 32, 10, 7, 2, 3, 4, 4], [2, 13, 2, 37, 16, 8, 20, 2], [29, 6, 13, 5, 1, 7, 7, 8, 20, 3, 1], [38, 9, 47, 5, 1], [7, 8, 26, 5, 2, 44, 5, 3], [7, 15, 21, 20, 16, 11, 4, 4, 2], [9, 10, 13, 5, 13, 6, 22, 10, 3, 8, 1], [1, 8, 22, 5, 56, 1, 1, 6], [74, 20, 2, 3, 1], [14, 31, 18, 34, 3], [38, 6, 2, 18, 1, 3, 26, 6], [9, 10, 46, 7, 4, 22, 1, 1], [1, 29, 56, 14], [68, 6, 1, 3, 9, 1, 2, 3, 5, 2], [15, 45, 8, 13, 6, 3, 2, 3, 4, 1], [2, 18, 1, 12, 25, 5, 19, 3, 9, 5, 1], [2, 10, 19, 37, 2, 4, 6, 3, 4, 2, 11], [11, 5, 26, 58], [15, 80, 5], [6, 36, 9, 4, 32, 7, 5, 1], [18, 7, 2, 27, 9, 4, 24, 4, 4, 1], [19, 6, 2, 4, 22, 1, 3, 40, 3], [26, 20, 2, 3, 47, 2], [16, 21, 1, 18, 15, 2, 26, 1], [52, 20, 28

## `Hill Climbing:`

In [115]:
import random

def hill_climbing_cutting_stock(stock_length, requests, max_iterations=1000, random_restart=False):
    # check if a request can fit in a roll
    def can_fit_in_roll(roll, request):
        return sum(roll) + request <= stock_length

    # generating an initial random solution
    def generate_initial_solution():
        random.shuffle(requests)  # Shuffle the requests for a random initial solution
        rolls = []
        for request in requests:
            placed = False
            for roll in rolls:
                if can_fit_in_roll(roll, request):
                    roll.append(request)
                    placed = True
                    break
            if not placed:
                rolls.append([request])
        return rolls

    # calculating the number of rolls used in a solution
    def calculate_rolls(rolls):
        return len(rolls)

    # generating neighboring solutions
    def get_neighbors(current_solution):
        neighbors = []
        for i in range(len(current_solution)):
            for j in range(len(current_solution)):
                if i != j:
                    for request in current_solution[i]:
                        if can_fit_in_roll(current_solution[j], request):
                            new_neighbor = [roll[:] for roll in current_solution]
                            new_neighbor[i].remove(request)
                            new_neighbor[j].append(request)
                            if not new_neighbor[i]:  # Remove empty roll
                                del new_neighbor[i]
                            neighbors.append(new_neighbor)
        return neighbors

    best_solution = None
    best_rolls = float('inf')

    for restart in range(random_restart + 1):
        current_solution = generate_initial_solution()
        current_rolls = calculate_rolls(current_solution)

        for _ in range(max_iterations):
            neighbors = get_neighbors(current_solution)
            improved = False

            for neighbor in neighbors:
                rolls = calculate_rolls(neighbor)
                if rolls < current_rolls:
                    current_solution = neighbor
                    current_rolls = rolls
                    improved = True
                    break

            if not improved:
                break

        if current_rolls < best_rolls:
            best_solution = current_solution
            best_rolls = current_rolls

        if not random_restart:
            break

    return best_solution, best_rolls





### `Overall Functionality:`

This code implements a Hill Climbing algorithm to solve the Cutting Stock Problem, focusing on reducing waste. Hill Climbing is a heuristic search algorithm that iteratively improves the solution by making small changes.


 ### `can_fit_in_roll` Function:
   - Checks if a request can fit into a roll without exceeding the stock length.

 ### `generate_initial_solution` Function:
- Creates a starting solution by randomly allocating orders to rolls.

 ### `calculate_rolls` Function:
 - Determines the number of rolls used in a solution. Fewer rolls indicate a better solution.

 ### `get_neighbors` Function:
   - Generates neighboring solutions by moving requests between rolls.

 ### `hill_climbing_cutting_stock` Function:
   - Manages the Hill Climbing process.
   - Starts with an initial solution and iteratively improves it by exploring neighboring solutions.
   - Can restart with a new random solution if enabled (`random_restart`).

 ### `best_solution` and `best_rolls` Tracking:
   - Tracks the best solution found and the number of rolls used in that solution.





### `Input Handling and Testing the Hill Climbing Algorithm:`

In [116]:
f = open('input1.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))

solution, rolls_used = hill_climbing_cutting_stock(stock_length, requests, random_restart=True)

# Outputting the result
print("Number of rolls used:", rolls_used)
print("Solution:", solution)

Number of rolls used: 50
Solution: [[123, 232, 351, 125, 60, 43, 61], [933, 18, 33], [501, 266, 92, 92, 45], [672, 249, 70], [662, 292, 46], [557, 301, 88, 53], [618, 346, 23], [581, 405], [544, 306, 125], [753, 106, 119], [689, 224, 78], [987], [368, 549, 80], [356, 517, 106], [753, 180], [506, 402, 88], [686, 312], [284, 532, 149], [211, 437, 337], [278, 518, 149], [555, 241, 106, 79], [557, 283, 144], [515, 118, 248, 107], [592, 354], [507, 412, 75], [632, 181, 170], [988], [788, 135, 71], [868, 99], [251, 268, 409, 69], [678, 286], [495, 126, 371], [805, 109, 86], [333, 187, 312, 145], [230, 653, 115], [295, 365, 286], [627, 266, 106], [967], [441, 414, 116], [660, 315], [463, 246, 117, 148], [788, 186], [525, 460], [218, 609, 84], [914], [370, 264, 149, 171], [424, 457], [648, 280], [557], [716]]


In [120]:
f = open('input2.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))

solution, rolls_used = hill_climbing_cutting_stock(stock_length, requests, random_restart=True)

# Outputting the result
print("Number of rolls used:", rolls_used)
print("Solution:", solution)

Number of rolls used: 78
Solution: [[2050, 1880, 1380], [2150, 2150], [1520, 1880, 2140], [2100, 1930, 1380], [2150, 2200], [2150, 1820, 1520], [1710, 2140, 1710], [1560, 2140, 1880], [2200, 1560, 1520], [2000, 2150, 1380], [2050, 1710, 1710], [1820, 2100, 1520], [2140, 2200], [1930, 1520, 2000], [1560, 1930, 1520], [2050, 1380, 2140], [2200, 2050], [1930, 1710, 1560], [2150, 1520, 1930], [2150, 2100], [2200, 1930, 1380], [2100, 1520, 1520], [2140, 2100], [1880, 1820, 1710], [2200, 1380, 1520], [1380, 2100, 1930], [2200, 2200], [2150, 1880, 1560], [1880, 1880, 1710], [2100, 2000, 1380], [2200, 1820, 1560], [1930, 1930, 1710], [1930, 2150, 1380], [2140, 2150], [2200, 2100], [2050, 2050, 1380], [1930, 2140, 1380], [2050, 2050, 1380], [2140, 1710, 1520], [2200, 1820, 1520], [2140, 1880, 1520], [1880, 2200, 1380], [2000, 2100, 1380], [2200, 2200], [1820, 1930, 1820], [1820, 1820, 1880], [2000, 2150, 1380], [2200, 1880, 1520], [1930, 1560, 1880], [2140, 2150], [2140, 1820, 1560], [1820, 152

In [118]:
f = open('input3.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))

solution, rolls_used = hill_climbing_cutting_stock(stock_length, requests, random_restart=True)

# Outputting the result
print("Number of rolls used:", rolls_used)
print("Solution:", solution)

Number of rolls used: 92
Solution: [[8, 2, 13, 263, 35, 8, 6, 1, 15, 11, 3, 21, 10, 1, 14, 11, 3, 3, 10, 3, 4, 11, 8, 2, 5, 4, 9, 4, 2, 4, 2, 3, 1], [138, 274, 18, 26, 11, 13, 13, 7], [111, 160, 118, 106, 2, 1, 2], [245, 255], [234, 87, 38, 138, 2, 1], [315, 180, 4, 1], [399, 7, 2, 4, 2, 39, 7, 7, 6, 5, 1, 2, 2, 2, 10, 3, 1, 1], [229, 245, 12, 11, 2, 1], [240, 178, 6, 5, 5, 12, 4, 6, 7, 6, 2, 9, 16, 3, 1], [298, 147, 15, 3, 13, 12, 7, 5], [167, 110, 111, 1, 20, 5, 8, 11, 9, 41, 14, 1, 2], [364, 66, 14, 6, 7, 4, 4, 4, 6, 12, 12, 1], [361, 16, 13, 16, 7, 76, 7, 3, 1], [237, 152, 17, 36, 3, 5, 2, 6, 7, 12, 2, 19, 1, 1], [290, 169, 13, 8, 9, 2, 7, 2], [319, 170, 6, 4, 1], [288, 170, 21, 15, 6], [311, 112, 9, 7, 7, 27, 10, 14, 1, 1, 1], [279, 201, 11, 4, 4, 1], [314, 128, 10, 8, 10, 17, 4, 9], [214, 280, 3, 3], [314, 153, 12, 16, 3, 2], [314, 108, 18, 9, 10, 4, 2, 3, 3, 11, 13, 2, 2, 1], [213, 183, 49, 37, 9, 7, 2], [126, 59, 12, 134, 21, 7, 8, 10, 72, 4, 18, 9, 6, 8, 3, 1, 2], [419, 28, 7,

In [121]:
f = open('input4.stock').readlines()
stock_length = int(f[0].split()[2])
requests = list(map(int, f[3].replace(' ','').split(',')))

solution, rolls_used = hill_climbing_cutting_stock(stock_length, requests, random_restart=True)

# Outputting the result
print("Number of rolls used:", rolls_used)
print("Solution:", solution)

Number of rolls used: 206
Solution: [[61, 34, 4, 1], [72, 8, 6, 8, 3, 2, 1], [18, 20, 18, 34, 10], [31, 36, 15, 10, 3, 3, 2], [20, 56, 21, 3], [54, 19, 27], [33, 60, 7], [51, 9, 20, 18, 2], [36, 33, 22, 8, 1], [31, 33, 13, 17, 6], [44, 34, 13, 9], [48, 20, 20, 7, 4, 1], [91, 6, 3], [74, 15, 4, 7], [46, 14, 33, 6, 1], [48, 19, 26, 4, 2, 1], [52, 17, 16, 13, 2], [53, 32, 14, 1], [78, 14, 7, 1], [16, 52, 16, 11, 5], [18, 19, 26, 26, 11], [38, 29, 15, 18], [37, 47, 7, 6, 3], [73, 20, 7], [66, 11, 6, 6, 9, 2], [93, 6, 1], [28, 26, 23, 13, 5, 5], [47, 34, 13, 6], [23, 39, 19, 14, 5], [47, 50, 3], [11, 27, 30, 22, 10], [35, 44, 10, 3, 4, 3, 1], [31, 24, 12, 16, 9, 6, 2], [98, 2], [51, 37, 10, 2], [27, 25, 36, 3, 3, 4, 2], [59, 21, 18, 2], [22, 16, 20, 34, 7, 1], [21, 30, 38, 7, 4], [24, 21, 22, 10, 22, 1], [89, 9, 2], [74, 20, 3, 3], [71, 15, 8, 4, 2], [44, 38, 10, 7, 1], [44, 11, 44, 1], [12, 75, 11, 2], [56, 22, 20, 2], [23, 62, 14, 1], [26, 6, 13, 24, 5, 6, 5, 3, 7, 5], [35, 27, 10, 13, 15